<a href="https://colab.research.google.com/github/Rayson-LD/flower-python/blob/main/Flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os

In [ ]:
url = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
zip_file = tf.keras.utils.get_file(origin=url,fname='flower_photos.tgz',extract=True,cache_subdir='/content')
base_dir = os.path.join(os.path.dirname(zip_file),'flower_photos')

228827136/228813984 [==============================] - 2s 0us/step


In [ ]:
image_size = 224
batch_size = 64
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (image_size,image_size),
    batch_size = batch_size,
    subset = 'training'
)
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (image_size,image_size),
    batch_size = batch_size,
    subset = 'validation'
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [ ]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt','w') as f :
  f.write(labels)

In [ ]:
image_shape = (image_size,image_size,3)
base_model = tf.keras.applications.MobileNetV2(input_shape = image_shape, include_top = False,weights ='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential(
    [
     base_model,
     tf.keras.layers.Conv2D(32,3,activation = 'relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.GlobalAveragePooling2D(),
     tf.keras.layers.Dense(5,activation = 'softmax'),
    ]
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs = 10

In [ ]:
history = model.fit(train_generator,
                    epochs = epochs,
                    validation_data = val_generator)

Epoch 1/10
46/46 [==============================] - 121s 3s/step - loss: 0.7145 - accuracy: 0.7547 - val_loss: 0.4573 - val_accuracy: 0.8413
Epoch 2/10
46/46 [==============================] - 115s 2s/step - loss: 0.2984 - accuracy: 0.8966 - val_loss: 0.4544 - val_accuracy: 0.8564
Epoch 3/10
46/46 [==============================] - 114s 2s/step - loss: 0.1894 - accuracy: 0.9326 - val_loss: 0.4674 - val_accuracy: 0.8536
Epoch 4/10
46/46 [==============================] - 114s 2s/step - loss: 0.1106 - accuracy: 0.9622 - val_loss: 0.4881 - val_accuracy: 0.8454
Epoch 5/10
46/46 [==============================] - 113s 2s/step - loss: 0.0723 - accuracy: 0.9775 - val_loss: 0.4906 - val_accuracy: 0.8468
Epoch 6/10
46/46 [==============================] - 113s 2s/step - loss: 0.0505 - accuracy: 0.9860 - val_loss: 0.4992 - val_accuracy: 0.8659
Epoch 7/10
46/46 [==============================] - 113s 2s/step - loss: 0.0285 - accuracy: 0.9949 - val_loss: 0.5734 - val_accuracy: 0.8550
Epoch 8/10
46

In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f :
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /tmp/tmp4ijo3yih/assets


INFO:tensorflow:Assets written to: /tmp/tmp4ijo3yih/assets


In [ ]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')

FileNotFoundError: ignored